In [4]:
import numpy as np
import pandas as pd
import random
from oandapyV20 import API
from sklearn.preprocessing import RobustScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import log_loss
import websocket
import threading
import time
import json

# ============== CONFIGURATION ==============
API_KEY = "60437d0eaa7fc1b272b4187bcb76715e-f4ec6754f1b275e59224b838534c7e37"
ACCOUNT_ID = "101-001-31838347-001"
TP_SL_RATIO = 2.0
TRAILING_STOP = 0.0035
DYNAMIC_WEIGHTING = True
COMPOUND_LOT = True
INSTRUMENT_ROTATION_INTERVAL = 900  # 15 minutes
MAX_CONCURRENT_TRADES = 10
RISK_PER_TRADE = 0.01  # 1% of equity

# ========== ASSET CLASS DEFINITIONS =========
ASSET_CATEGORIES = {
    "Forex": {
        "Major": ["EUR_USD", "USD_JPY", "GBP_USD"],
        "Minor": ["EUR_GBP", "AUD_JPY", "NZD_CAD"]
    },
    "Crypto": ["BTC_USD", "ETH_USD", "LTC_USD", "BCH_USD"],
    "Commodities": ["XAU_USD", "XAG_USD", "BCO_USD"],
    "Indices": ["US30_USD", "UK100_GBP", "DE30_EUR"],
    "Shares": ["AAPL_USD", "MSFT_USD", "TSLA_USD"],
    "ETFs": ["SPY_USD", "GLD_USD", "QQQ_USD"]
}

# ========== DYNAMIC INSTRUMENT MANAGER ==========
class InstrumentRotator:
    def __init__(self, api_client):
        self.api = api_client
        self.active_instruments = []
        self.last_rotation = 0
        self.volatility_cache = {}
        
    def _get_activity_score(self, instrument):
        """Calculate volatility-volume composite score"""
        # Use cached value if recent
        if instrument in self.volatility_cache and time.time() - self.volatility_cache[instrument]['timestamp'] < 300:
            return self.volatility_cache[instrument]['score']
        
        # Calculate fresh score
        data = self.api.get_history(instrument, count=100, granularity="M5")
        volatility = data['close'].pct_change().std()
        volume = data['volume'].mean()
        score = volatility * volume
        
        # Update cache
        self.volatility_cache[instrument] = {
            'score': score,
            'timestamp': time.time()
        }
        return score
        
    def rotate_instruments(self):
        if time.time() - self.last_rotation < INSTRUMENT_ROTATION_INTERVAL:
            return
            
        all_instruments = []
        for category, instruments in ASSET_CATEGORIES.items():
            if isinstance(instruments, dict):
                for subcat in instruments.values():
                    all_instruments.extend(subcat)
            else:
                all_instruments.extend(instruments)
                
        scored = []
        for inst in all_instruments:
            try:
                score = self._get_activity_score(inst)
                scored.append((inst, score))
            except:
                continue
                
        scored.sort(key=lambda x: x[1], reverse=True)
        self.active_instruments = [inst[0] for inst in scored[:MAX_CONCURRENT_TRADES]]
        self.last_rotation = time.time()

# ========== PROBABILISTIC MODEL (RF with Uncertainty) ==========
class ProbabilisticModel:
    def __init__(self, n_estimators=100):
        self.model = RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=5,
            random_state=42,
            class_weight='balanced'
        )
        
    def train(self, X, y):
        self.model.fit(X, y)
        
    def predict_with_uncertainty(self, X):
        predictions = self.model.predict_proba(X)
        # Use entropy as uncertainty measure
        uncertainty = np.array([log_loss([1,0], p, normalize=False) for p in predictions])
        return predictions[:, 1], uncertainty

# ========== DYNAMIC CAPITAL ALLOCATOR ==========
class PerformanceWeighter:
    def __init__(self):
        self.instrument_performance = {}
        self.weights = {}
        
    def update_performance(self, instrument, pnl, volatility):
        if instrument not in self.instrument_performance:
            self.instrument_performance[instrument] = {'returns': [], 'vols': []}
            
        self.instrument_performance[instrument]['returns'].append(pnl)
        self.instrument_performance[instrument]['vols'].append(volatility)
        
        # Maintain rolling window of 100 trades
        if len(self.instrument_performance[instrument]['returns']) > 100:
            self.instrument_performance[instrument]['returns'].pop(0)
            self.instrument_performance[instrument]['vols'].pop(0)
            
    def calculate_weights(self):
        scores = {}
        for inst, data in self.instrument_performance.items():
            returns = np.array(data['returns'])
            vols = np.array(data['vols'])
            score = np.mean(returns) / (np.mean(vols) + 1e-9)
            scores[inst] = max(score, 0)  # No negative weights
            
        total = sum(scores.values())
        if total > 0:
            self.weights = {k: v/total for k, v in scores.items()}
        else:
            # Equal weighting if no performance data
            n = len(scores)
            self.weights = {k: 1/n for k in scores.keys()} if n > 0 else {}

# ========== MAIN TRADING ENGINE ==========
class QuantumTrader:
    def __init__(self):
        self.api = API(access_token=API_KEY)
        self.account_id = ACCOUNT_ID
        self.rotator = InstrumentRotator(self.api)
        self.weighter = PerformanceWeighter()
        self.models = {}
        self.active_positions = {}
        self.equity = self.api.get_account_summary()['balance']
        self.data_buffer = {inst: [] for inst in self.get_all_instruments()}
        
        # Initialize models for all instruments
        for inst in self.get_all_instruments():
            self.models[inst] = ProbabilisticModel()
        
        # Start WebSocket stream
        self.ws_thread = threading.Thread(target=self._start_websocket)
        self.ws_thread.daemon = True
        self.ws_thread.start()
        
    def get_all_instruments(self):
        instruments = []
        for category, items in ASSET_CATEGORIES.items():
            if isinstance(items, dict):
                for subcat in items.values():
                    instruments.extend(subcat)
            else:
                instruments.extend(items)
        return instruments
        
    def _start_websocket(self):
        def on_message(ws, message):
            data = json.loads(message)
            self.process_real_time_data(data)
            
        ws = websocket.WebSocketApp(
            f"wss://stream-fxtrade.oanda.com/v3/accounts/{ACCOUNT_ID}/pricing/stream",
            on_message=on_message
        )
        ws.run_forever()
        
    def process_real_time_data(self, data):
        # Update prices and check triggers
        for instrument in data['prices']:
            inst_id = instrument['instrument']
            bid = float(instrument['bids'][0]['price'])
            ask = float(instrument['asks'][0]['price'])
            mid = (bid + ask)/2
            
            if inst_id in self.active_positions:
                self._update_trailing_stop(inst_id, bid, ask)
                
            # Store data for model processing
            self.data_buffer[inst_id].append(mid)
            if len(self.data_buffer[inst_id]) > 100:
                self.data_buffer[inst_id].pop(0)
        
    def _update_trailing_stop(self, instrument, bid, ask):
        position = self.active_positions[instrument]
        current_price = bid if position['type'] == 'sell' else ask
        
        if position['type'] == 'buy':
            new_stop = current_price * (1 - TRAILING_STOP)
            if new_stop > position['stop_loss']:
                self.api.update_order(position['id'], stop_loss=new_stop)
                position['stop_loss'] = new_stop
        else:
            new_stop = current_price * (1 + TRAILING_STOP)
            if new_stop < position['stop_loss']:
                self.api.update_order(position['id'], stop_loss=new_stop)
                position['stop_loss'] = new_stop
                
    def generate_signals(self):
        self.rotator.rotate_instruments()
        for instrument in self.rotator.active_instruments:
            # Skip if no data
            if len(self.data_buffer[instrument]) < 30:
                continue
                
            # Prepare features and target
            X, y = self.feature_engineer(instrument)
            
            # Train and predict
            self.models[instrument].train(X[:-1], y[:-1])
            proba, uncertainty = self.models[instrument].predict_with_uncertainty(X[-1:])
            
            # Confidence-based sizing
            confidence = max(0, 1 - uncertainty[0]/np.log(2))  # Normalize entropy
            
            if proba[0] > 0.55 and instrument not in self.active_positions:
                self.enter_trade(instrument, 'buy', confidence)
            elif proba[0] < 0.45 and instrument not in self.active_positions:
                self.enter_trade(instrument, 'sell', confidence)
                
    def enter_trade(self, instrument, direction, confidence):
        # Get current price
        price_data = self.api.get_prices(instruments=instrument)
        bid = float(price_data['prices'][0]['bids'][0]['price'])
        ask = float(price_data['prices'][0]['asks'][0]['price'])
        mid = (bid + ask)/2
        
        # Calculate stop and target
        if direction == 'buy':
            stop_loss = mid * (1 - TRAILING_STOP)
            take_profit = mid * (1 + TP_SL_RATIO * TRAILING_STOP)
        else:
            stop_loss = mid * (1 + TRAILING_STOP)
            take_profit = mid * (1 - TP_SL_RATIO * TRAILING_STOP)
        
        # Calculate position size
        risk_amount = self.equity * RISK_PER_TRADE * confidence
        if DYNAMIC_WEIGHTING:
            risk_amount *= self.weighter.weights.get(instrument, 1.0)
            
        price_risk = abs(mid - stop_loss)
        units = risk_amount / price_risk
        
        # Execute trade
        order = {
            "order": {
                "instrument": instrument,
                "units": str(int(units)) if direction == 'buy' else str(-int(units)),
                "type": "MARKET",
                "stopLoss": str(round(stop_loss, 5)),
                "takeProfit": str(round(take_profit, 5))
            }
        }
        response = self.api.create_order(ACCOUNT_ID, order)
        self.active_positions[instrument] = {
            'id': response['orderID'],
            'type': direction,
            'stop_loss': stop_loss,
            'entry': mid,
            'size': units
        }
        
    def feature_engineer(self, instrument):
        """Create multi-timeframe features"""
        prices = np.array(self.data_buffer[instrument])
        returns = np.diff(prices) / prices[:-1]
        
        # Feature matrix
        features = []
        
        # Momentum features
        features.append(returns[-5:])       # Recent returns
        features.append([np.mean(returns[-20:])  # Medium-term momentum
        features.append([np.mean(returns[-50:])  # Long-term momentum
        
        # Volatility features
        features.append([np.std(returns[-10:])   # Short-term vol
        features.append([np.std(returns[-30:])   # Medium-term vol
        
        # Technical patterns
        high = np.max(prices[-10:])
        low = np.min(prices[-10:])
        features.append([(prices[-1] - low) / (high - low + 1e-9)])  # % range position
        
        # Create labels (1 if next return positive, 0 otherwise)
        y = (returns[1:] > 0).astype(int)
        
        # Align features and labels
        X = np.vstack(features).T[:-1]  # Exclude last point since no future return
        y = y[:len(X)]  # Ensure same length
        
        return X, y

# ========== RUN SYSTEM ==========
if __name__ == "__main__":
    trader = QuantumTrader()
    
    while True:
        try:
            trader.generate_signals()
            # Update equity from account
            trader.equity = trader.api.get_account_summary()['balance']
            # Update weights
            trader.weighter.calculate_weights()
            time.sleep(60)  # Check signals every minute
        except Exception as e:
            print(f"Error: {e}")
            # Fallback to REST API if WebSocket fails
            prices = trader.api.get_prices(instruments=",".join(trader.rotator.active_instruments))
            trader.process_real_time_data(prices)

SyntaxError: invalid syntax. Perhaps you forgot a comma? (2911915749.py, line 288)